# HomeWork 3


Домашнее задание: максимум 20 баллов

1. Побейте метрику на лидерборде map@10 = 0.063 для userKnn модели с семинара (5 баллов)

2. Реализуйте эксперименты с кастомной моделю kNN с семинара. Результат - ноутбук(и) (максимум 15 баллов)

(Вы можете отрефакторить код из userknn.py по желанию или не трогать его)

Что можно сделать в ноутбуке:

- придумать, что делать с холодными пользователями в тесте. 
- Сделайте рекомендации для них (обратите внимание на rectools.models.popular) (3 балла)
  - UserPopular
  - UserPopularTopCategory
  - NewItems
- Сделать кол-во рекомендаций равным N, а не меньше N (3 балла)
  - Popular
  - NewItems
  - ?LastTopViewed
- Реализовать тюнинг гиперпараметров (например, векторного расстояния или типов kNN моделей (implicit/rectools/...)) и сделать выводы (3 балла)
  - реализовать другие варианты ранжированивания айтемов похожих пользователей и сделать выводы (3 балла)
  - провести эксперименты с параметрами оффлайн валидации и сделать выводы (3 балла)
    - Optuna
    - Wandb, ClearML, MetaFlow

1. Оберните модель в сервис (максимум 12 баллов)

    предпочтительный онлайн вариант: 
    - обучаете модель в ноутбуке, 
    - сохраняете обученную модель (pickle, dill), 
    - при запуске сервиса ее поднимаете и запрашиваете рекомендации "на лету" (12 баллов)
    или оффлайн вариант: 
    - предварительно посчитайте рекомендации для всех пользователей
    - сохраните и запрашивайте их (6 баллов)

Хороший pull request - это:  
    - наличие описания (в идеале что сделано - по пунктам)  
    - код по стандарту PEP8  
    - легкая читаемость и воспроизводимость кода  
    - комментарии и объяснения. В ipynb пользуйтесь силой маркдауна. В скриптах пишите комментарии и докстринг.  
    - обоснование схемы валидации  
    - анализ метрики качества  


In [1]:
import pandas as pd
import numpy as np

#!pip install rectools
#!pip install seaborn
#!pip install pyarrow
# !pip install Jinja2

In [2]:
inter = pd.read_csv(
    "../data/interactions.csv",
)

items = pd.read_csv(
    '../data/items.csv',
)

# users = pd.read_csv(
#     '../data/users.csv',
# )

In [3]:
inter.head(3)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0


In [4]:
inter = inter.rename(columns={"last_watch_dt": "datetime"})

inter["datetime"] = pd.to_datetime(inter["datetime"])

inter["weight"] = inter["total_dur"] * inter["watched_pct"]
# inter['weight'] = np.where(inter['watched_pct'] > 25, 1, 0) * inter['total_dur']

# Задание 1

In [5]:
from rectools.metrics import (
    Precision,
    Recall,
    NDCG,
    MAP,
    Serendipity,
    MeanInvUserFreq,
    calc_metrics,
)

from implicit.nearest_neighbours import TFIDFRecommender
from rectools.models import RandomModel, PopularModel, PopularInCategoryModel, ImplicitItemKNNWrapperModel
from rectools import Columns
from rectools.model_selection.time_split import TimeRangeSplitter
from rectools.dataset import Interactions, Dataset

/home/kartushov/pet_projects/itmo/itmo.recsys/RecoServiceTemplate/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kartushov/pet_projects/itmo/itmo.recsys/RecoServiceTemplate/.venv/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [6]:
K = 10

n_splits = 3

splitter = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

models = {
    "random": RandomModel(random_state=42),
    "popular": PopularModel(),
    "itemKNN": ImplicitItemKNNWrapperModel(TFIDFRecommender(K=K)),
}

metrics = {
    "MAPTop-1": MAP(k=1),
    "MAPTop-5": MAP(k=5),
    "MAPTop-10": MAP(k=10),
    "NDCG@1": NDCG(k=1),
    "NDCG@5": NDCG(k=5),
    "NDCG@10": NDCG(k=10),
    "Precision@1": Precision(k=1),
    "Precision@5": Precision(k=5),
    "Precision@10": Precision(k=10),
    "Recall@1": Recall(k=1),
    "Recall@5": Recall(k=5),
    "Recall@10": Recall(k=10),
    "Novelty@1": MeanInvUserFreq(k=1),
    "Novelty@5": MeanInvUserFreq(k=5),
    "Novelty@10": MeanInvUserFreq(k=10),
    "Serendipity": Serendipity(k=1),
    "Serendipity": Serendipity(k=5),
    "Serendipity": Serendipity(k=10),
}


In [7]:
import time
import pickle
from tqdm import tqdm


class RecSysModule:
    def __init__(
        self, model_dict: dict = None, metrics_dict: dict = None, splitter: TimeRangeSplitter = None, k: int = 10
    ):
        """
        Initialize the RecSysModule object.

        Parameters
        ----------
        model_dict : dict, optional
            Dictionary containing models, by default None
        metrics_dict : dict, optional
            Dictionary containing metrics, by default None
        splitter : TimeRangeSplitter, optional
            TimeRangeSplitter object for splitting data, by default None
        k : int, optional
            Value of 'k' for top-k recommendations, by default 10

        Raises
        ------
        ValueError
            If splitter is not provided.
        ValueError
            If metrics_dict is not provided.
        ValueError
            If model_dict is not provided.
        ValueError
            If no metrics are provided.
        ValueError
            If no models are provided.
        ValueError
            If k is negative.
        """
        if splitter is None:
            raise ValueError("Please provide splitter")
        if metrics_dict is None:
            raise ValueError("Please provide metrics_dict")
        if model_dict is None:
            raise ValueError("Please provide model_dict")
        if len(metrics_dict) == 0:
            raise ValueError("Please provide at least one metric")
        if len(model_dict) == 0:
            raise ValueError("Please provide at least one model")
        if k < 0:
            raise ValueError("k cannot be negative")

        self.model_dict = model_dict
        self.metrics_dict = metrics_dict
        self.splitter = splitter
        self.k = k

    def fold_iterator(self, dataframe: pd.DataFrame, treshold=5):        
        count_views_items = dataframe.groupby('user_id')['item_id'].count()
        train_ids = list(count_views_items[count_views_items > treshold].index)
        test_ids = list(count_views_items[count_views_items <= treshold].index)

        test_ids = [user_id for user_id in test_ids if user_id in train_ids]

        return train_ids, test_ids

    def evaluate(self, interaction: pd.DataFrame) -> pd.DataFrame:
        results = []
        train_ids, test_ids = self.fold_iterator(interaction.df)
 
        start_time = time.time()

        train = interaction.df.iloc[train_ids]
        test = interaction.df.iloc[test_ids][Columns.UserItem]

        dataset = Dataset.construct(train)
        test_users = np.unique(test[Columns.User])
        catalog = train[Columns.Item].unique()

        for model_name, model in models.items():
            model.fit(dataset)
                
            with open(f'../models/{model_name}.pickle', 'wb') as f:
                pickle.dump(model, f)

            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=self.k,
                filter_viewed=True,
            )

            metric_values = calc_metrics(
                    metrics,
                    reco=recos,
                    interactions=test,
                    prev_interactions=train,
                    catalog=catalog,
                )
            end_time = time.time()
            evaluation_time = end_time - start_time

            res = {"model": model_name, "evaluation_time": evaluation_time}
            res.update(metric_values)

        self.metrics_result = results

    def visualize_results(self):
            pivot_results = pd.DataFrame(self.metrics_result).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])
            mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
            return (
                pivot_results.style
                .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
                .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
            )

    def visualize_analysis(self, model, dataset, user_ids, item_data):
        recos = model.recommend(users=user_ids, dataset=dataset, k=self.k, filter_viewed=True)
        recos = recos[recos[Columns.User].isin(user_ids)]
        recos = recos.merge(item_data, on=[Columns.Item], how='left')[['title', 'genres', Columns.User, Columns.Item]]
        
        history = dataset.interactions.df
        history = history[history[Columns.User].isin(user_ids)]
        history = history.merge(item_data, on=[Columns.Item], how='left')[['title', 'genres', Columns.User, Columns.Item]]

        for user in user_ids:
            print(f"User {user} history:")
            history_items = history[Columns.Item].unique()
            print(f"Total count views user: {history[history[Columns.User] == user][Columns.Item].nunique()}")
            for item in history_items:
                _his_item_dataset = dataset.interactions.df
                item_title = history[history[Columns.Item] == item]['title'].values[0]
                # unique equal count cuz 1 user ~ 1 item in this set
                item_unviews = _his_item_dataset[_his_item_dataset[Columns.Item] == item][Columns.User].nunique()
                print(f"{item_title} | total views: {item_unviews}")
            display(history[history[Columns.User] == user])
            print()
            print(f"User {user} recommendations:")
            display(recos[recos[Columns.User] == user])
            print()
            print()

In [8]:
interactions = Interactions(inter)

recsys = RecSysModule(
    model_dict=models,
    metrics_dict=metrics,
    splitter=splitter,
    k=10,
)

In [9]:
recsys.evaluate(interactions)

In [10]:
recsys.visualize_results()

In [17]:
with open('../models/itemKNN_0.pickle', 'rb') as f:
    random_model = pickle.load(f)

users2test = [666262, 672861, 955527]

dataset = Dataset.construct(inter)

recsys.visualize_analysis(
    random_model,
    dataset=dataset,
    user_ids=users2test,
    item_data=items,
)

User 666262 history:
Total count views user: 1
Дом ночных призраков | total views: 68581
Медвежонок Винни и его друзья | total views: 202457
В ритме сердца | total views: 132865
Признание 5 | total views: 193123


,title,genres,user_id,item_id
0,Дом ночных призраков,"зарубежные, криминал, детективы, ужасы",666262,93



User 666262 recommendations:


,title,genres,user_id,item_id
0,Рэкетир 2,"драмы, боевики, криминал",666262,3194
1,Кутерьма,"историческое, биография, драмы, мюзиклы, комедии",666262,3520
2,Хорнблауэр,"драмы, зарубежные, военные, приключения",666262,3248
3,Логово монстра,"зарубежные, триллеры, ужасы",666262,3136
4,Коммуналка,"драмы, зарубежные",666262,12757
5,Уайат Сенак разрулит,документальное,666262,3965
6,Барбос в гостях у Бобика,"советские, комедии",666262,10790
7,Атака легкой кавалерии,"боевики, военные, приключения, мелодрамы",666262,5581
8,Вложения,"зарубежные, комедии, экранизации",666262,9454
9,Лига чемпионов ФИБА. Нижний Новгород (Россия) ...,спорт,666262,11133




User 672861 history:
Total count views user: 2
Дом ночных призраков | total views: 68581
Медвежонок Винни и его друзья | total views: 202457
В ритме сердца | total views: 132865
Признание 5 | total views: 193123


...

,title,genres,user_id,item_id
10,Джуди,"драмы, биография, музыкальные, мелодрамы",672861,5545
11,Маленькие лесные истории,"мультсериалы, приключения",672861,4076
12,Скопа,драмы,672861,8088
13,Детективы,детективы,672861,7315
14,Цвета любви,"драмы, мелодрамы",672861,2906
15,[4k] Соник в кино,"фильмы, приключения, зарубежные, фантастика, б...",672861,15615
16,Психонавт,"драмы, зарубежные, фантастика, музыкальные",672861,8164
17,Смерть Эйприл,"зарубежные, детективы, ужасы",672861,11007
18,Веном,"популярное, фантастика, триллеры, боевики, ужасы",672861,14317
19,Истеричка,драмы,672861,394




User 955527 history:
Total count views user: 1
Дом ночных призраков | total views: 68581
Медвежонок Винни и его друзья | total views: 202457
В ритме сердца | total views: 132865
Признание 5 | total views: 193123


,title,genres,user_id,item_id
3,Признание 5,для взрослых,955527,21



User 955527 recommendations:


,title,genres,user_id,item_id
20,Любовь,"драмы, мелодрамы",955527,14470
21,Чтец,"драмы, мелодрамы",955527,8022
22,Викинг Вик,"боевики, мультфильм, приключения, комедии",955527,11717
23,Светофор,комедии,955527,2036
24,Люди в чёрном 2,"приключения, детективы, фантастика, боевики, к...",955527,6577
25,В плену обмана,"русские, мелодрамы",955527,464
26,Веселые формы,для взрослых,955527,122
27,"Ронья, дочь разбойника","для детей, аниме, сериалы, приключения, зарубе...",955527,15334
28,Если только,"драмы, мелодрамы, фэнтези, комедии",955527,10176
29,Под Большой медведицей,"русские, мелодрамы",955527,9888


In [31]:
with open('../models/random_0.pickle', 'rb') as f:
    random_model = pickle.load(f)

users2test = [666262, 672861, 955527]

dataset = Dataset.construct(inter)

recsys.visualize_analysis(
    random_model,
    dataset=dataset,
    user_ids=users2test,
    item_data=items,
)

User 666262 history:
Total count views user: 4
Дом ночных призраков | total views: 68581 | nunique users: 68581
Медвежонок Винни и его друзья | total views: 202457 | nunique users: 202457
В ритме сердца | total views: 132865 | nunique users: 132865
Признание 5 | total views: 193123 | nunique users: 193123


,title,genres,user_id,item_id
0,Дом ночных призраков,"зарубежные, криминал, детективы, ужасы",666262,93



User 666262 recommendations:


,title,genres,user_id,item_id
0,Ода радости,комедии,666262,7419
1,Последняя битва,"драмы, военные",666262,9109
2,Преисподняя,"драмы, детективы, триллеры, вестерн",666262,13917
3,Лихорадка,ужасы,666262,13332
4,Вечность,драмы,666262,1331
5,Леший,"триллеры, криминал, детективы",666262,15448
6,Богема,"драмы, мюзиклы, мелодрамы",666262,1123
7,Шедевр,"драмы, комедии",666262,9933
8,Единоборства для детей (3-6 лет) Школа героев ...,"единоборства, фитнес, для детей",666262,3287
9,Американский пирог: Все в сборе,комедии,666262,8478




User 672861 history:
Total count views user: 4
Дом ночных призраков | total views: 68581 | nunique users: 68581
Медвежонок Винни и его друзья | total views: 202457 | nunique users: 202457
В ритме сердца | total views: 132865 | nunique users: 132865
Признание 5 | total views: 193123 | nunique users: 193123


...

,title,genres,user_id,item_id
10,Джек: Путь смерти,"боевики, зарубежные, триллеры, криминал",672861,1483
11,Фиаско,комедии,672861,11062
12,Песня Любви (субтитры),"драмы, мелодрамы",672861,12863
13,Палата,"зарубежные, триллеры, ужасы",672861,15487
14,Перспектива (с тифлокомментарием),"фантастика, драмы, зарубежные, триллеры",672861,1818
15,Красотка,"мелодрамы, комедии",672861,14809
16,Смешарики.Легенда о золотом драконе,"мультфильм, приключения, комедии",672861,8207
17,Тихоокеанский рубеж 2,"боевики, фантастика, приключения",672861,8799
18,Расти-механик,"семейное, для детей",672861,14725
19,Учим математику с Ам Нямом,"русские, для детей, развитие, сериалы",672861,11960




User 955527 history:
Total count views user: 4
Дом ночных призраков | total views: 68581 | nunique users: 68581
Медвежонок Винни и его друзья | total views: 202457 | nunique users: 202457
В ритме сердца | total views: 132865 | nunique users: 132865
Признание 5 | total views: 193123 | nunique users: 193123


,title,genres,user_id,item_id
3,Признание 5,для взрослых,955527,21



User 955527 recommendations:


,title,genres,user_id,item_id
20,"Леопард, великолепная способность к адаптации....",документальное,955527,11405
21,Творение Господне,"драмы, биография, фильмы hbo",955527,14305
22,Мусульмане как мы,документальное,955527,7720
23,Проклятие ведьмы снов,"зарубежные, фантастика, триллеры, боевики, ужа...",955527,1212
24,Искусство путешествовать,"зарубежные, приключения",955527,8485
25,Мозаика,"драмы, детективы",955527,12234
26,Мороз по коже,мелодрамы,955527,12862
27,Класс коррекции,драмы,955527,6604
28,Договор молчания,"триллеры, детективы",955527,5291
29,Третья волна зомби,"фантастика, зарубежные, драмы, ужасы",955527,6538
